In [2]:
#import necessary libraries
from scipy.stats import unitary_group
import math
import numpy as np
from numpy import linalg as LA
import itertools as it
import qiskit as qk
# from qiskit import Aer, IBMQ, execute
import matplotlib.pyplot as plt

In [3]:
#Define simulation parameters
N = 3 #number of spins/qubits
#TFIM Simulation parameters
Jx = 0.01183898
hz = 2.0*Jx
freq = 0.0048
delta_t = 3 #time-step size
num_steps = 40 #number of time-steps, we start with a small number for speed of demonstration
shots = 8192 #number of shots for circuit execution


In [4]:
#create the circuit to execute the time-evolution operator for a given time-step
def evolution_circuit_TFIM(num_time_steps, Jx, hz, freq, delta_t, N):
    hbar = 0.658212    # eV*fs
    time_evol_circuit = qk.QuantumCircuit(N)
    #define rotation angles for gates in circuit
    psiX = -2.0*Jx*delta_t/hbar
    for step in range(num_time_steps):
        t = (step + 0.5) * delta_t
        psiZ = -2.0*hz*np.cos(2*np.pi*freq*t)*delta_t/hbar
        #implement XX operator
        for q in range(0,N-1):
            time_evol_circuit.h(q)
            time_evol_circuit.h(q+1)
            time_evol_circuit.cx(q,q+1)
            time_evol_circuit.rz(psiX,q+1)
            time_evol_circuit.cx(q,q+1)
            time_evol_circuit.h(q)
            time_evol_circuit.h(q+1)
        #implement Z operator
        for q in range(0,N):
            time_evol_circuit.rz(psiZ, q)
    return time_evol_circuit


In [5]:
#Create set of final circuits for quantum quench simulations
circuits = []
for i in range(0, num_steps+1):
    #TFIM
    circuits.append(evolution_circuit_TFIM(i,Jx,hz,freq,delta_t,N))

In [ ]:
from bqskit.ir.gates import *

def make_matchgate():
    rz = ZGate()
    rx = XGate()
    cnot = CNOTGate()
    I = IdentityGate()

    RZ_layer = KroneckerGate(rz,rz)
    RX_layer = KroneckerGate(rx,rx)
    RXRZ_layer = KroneckerGate(rx, rz)
    matchgate = ProductGate(RZ_layer, RX_layer, cnot, RXRZ_layer, cnot, RX_layer, RZ_layer)
    return matchgate
    
def make_layertype1(N):
    I = IdentityGate()
    for i in range(int(N/2)):
        if (i==0):
            layer = make_matchgate()
        else:
            layer = KroneckerGate(layer,make_matchgate())
    if (N%2 != 0): 
        layer = KroneckerGate(layer, I)
    return layer
        
def make_layertype2(N):
    I = IdentityGate()
    layer = I
    #N even
    if (N%2==0):
        for _ in range(int(N/2)-1):
            layer = KroneckerGate(layer,make_matchgate())
        layer = KroneckerGate(layer,I)
    #N odd
    else:
        for _ in range(int(N/2)):
            layer = KroneckerGate(layer,make_matchgate())
    return layer

def make_MGC(N):
    """
    Make matchgate circuit for N qubits.
    Args:
        N (int): The number of spins.
    Returns
        circuit (ProductGate): Circuit of QSearch gates.
    """
    for l in range(N):
        #add layer_type1
        if (l%2 == 0):
            if(l==0):
                circuit = make_layertype2(N)
            else:
                circuit = ProductGate(circuit, make_layertype2(N))
        #add layer_type2
        else:
            circuit = ProductGate(circuit, make_layertype1(N))
    return circuit